In [5]:
import os, json
from letta import create_client 
from letta import EmbeddingConfig, LLMConfig
import letta

client = create_client()
client.set_default_embedding_config(EmbeddingConfig.default_config(model_name="letta"))
client.set_default_llm_config(LLMConfig.default_config(model_name="letta"))

# client.send_message(
#     agent_id="agent-2373b6c6-8fbb-4e1c-bfce-c027c3b17530",
#     message=""
# )



In [ ]:
import letta.data_sources
import letta.data_sources.connectors
import json

def compareSimilarities(path):
    """
    This tool sorts the user's file system.
    If asked to sort or to organize the file system, call this function.

    Args: 
    Returns: 
    """

    file = open(path)
    fileData = json.load(file)
    #fileDataS = json.dumps(fileData)
    return fileData

file_data = compareSimilarities('FileSystemData.json')
file_keys = file_data['TestDirectory']['children'].keys()
for k in file_keys:
    print(k)

In [96]:
import os, json
from letta import create_client 
from letta import EmbeddingConfig, LLMConfig
import letta
from letta import AgentState


def sort_FS(self, mode: str):
    """
    This tool sorts the user's file system.
    If asked to sort or to organize the file system, call this function.
    Tell the user the return and results of the function call.


    Args: 
        mode (str): This is the mode or method in which the user wants the sorting to be done. 

    Returns: 
        fileSort (str): This is the new organization of the file system users requested
    """
    #create another client to ask
    #Options are: content, extension, name, or date.
    # Query the user for the mode (valid modes are: content, extension, name, or date), before calling this function. 
    from letta import create_client 
    from letta import EmbeddingConfig, LLMConfig
    import json, os, re
    
    client = create_client()
    client.set_default_embedding_config(EmbeddingConfig.default_config(model_name="letta"))
    client.set_default_llm_config(LLMConfig.default_config(model_name="letta"))

    #get json data
    #file = open('FileSystemData.json') #NOTE: hardCoded
    file = open('dataTable3.json') #NOTE: hardCoded
    fileData = json.load(file)
    #fileDataS = json.dumps(fileData)

    #create message to send to different agent
    #NOTE: 1 layer simple sort, based on name
    file_keys = fileData.keys()

    
    template = "\nTemplate:\n[group 1]{item 1, item2, ...}\n[group 2]{item 3, item 4, ...}\n...\nItems:"

    enforceSingle = "If an item has already be grouped, do not group it again. Thus, for each item listed, it should only appear once amoungst all groups. DO NOT CHANGE THE FILE NAMES. Return only the output template and nothing else:"
    
    if mode == 'name':
        prompt = f"Using the template and given the following files, group the files into the template based on the commonalities in the topic, semantics, or meanings of the names. Ignore the file extension or file type"

        message = prompt + enforceSingle + template
        for k in file_keys:
            message = message + "\n" + k

    elif mode == 'extension':
        prompt = f"Using the template and given the following files, group the files into the template based on the commonalities of ONLY on the {mode}s of the file, and not on the name."

        message = prompt + enforceSingle + template
        for k in file_keys:
            message = message + "\n" + k
    elif mode  == 'content':
        #NOTE: when we say content, we are also refereing to file summary
        prompt = f"Using the template and given the following files, group the files into the template based on the commonalities of ONLY on the summary of the file."

        message = prompt + enforceSingle +template
        for k in file_keys:
            # print(k)
            try:
                message =  message + '\n file: "' + k + '" content: "' + fileData[k]['summary'] + '"'
                #TODO: maybe change content to summary 
            except KeyError:
                continue 
    
    

    print(f'Testing Output: {message}')

    #NOTE: for easier testing, try to start fresh so we know effect of prompting
    try:
        agentState = client.create_agent('background_agent')
    except ValueError:
        agentID = client.get_agent_id('background_agent')
        client.delete_agent(agentID)
        agentState = client.create_agent('background_agent')
    
    
    answer = client.send_message(
        message=message,
        role='user',
        agent_id=agentState.id
    )

    

    answer = answer.messages[1].function_call.arguments
    answer = json.loads(answer)
    answer = json.dumps(answer['message'])
    print("MY_ANSWER", answer)

    #self.interface.assistant_message(answer)  # , msg_obj=self._messages[-1])
    outputFile = open("newFileSystem.txt", 'w')
    outputFile.write(answer) #NOTE: Store background agent output

    #----ParsingGPT_FS----
    raw_folders = re.findall(r'(\[[\w\s\-\,\.\\\/\(\)]+\])', answer)
    # print(raw_folders)

    raw_files = re.findall(r'\{[\w\s\-\,\.\\\/\(\)]+\}', answer)
    # print(raw_files)

    #split and clean files
    files = []
    for f in raw_files:
        f = re.sub(r'[\{\}]', '', f)
        files.append(re.split(', ', f))
        # print(files)

    print('Files:  ', files)

    #clean folders
    folders = []
    for f in raw_folders:
        f = re.sub(r'[\[\]]', '', f)
        f = re.sub(r' ', '_', f)
        folders.append(re.split(', ', f)[0])

    print('Folders:  ', folders)

    #generate json file table
    fs_table = {}
    dir_root = 'TestDirectory3' #NOTE: Hard Coded
    for folder_i in range(len(folders)):
        for file_i in range(len(files[folder_i])):
            old_path = fileData[files[folder_i][file_i]]['path']

            fs_table[files[folder_i][file_i]] = {
                "type" : "file",
                "new_path" : f"{dir_root}/{folders[folder_i]}",
                "old_path" : old_path
            }
        
    print(json.dumps(fs_table, indent=4))
    #---------------------END


    #----CREATE_NEW_FS----
    import shutil
    for fileName in fs_table.keys():
        print('MOVING: ', fileName)
        try:
            os.mkdir(fs_table[fileName]['new_path'])
        except FileExistsError: 
            continue
        finally:
            shutil.move(fs_table[fileName]['old_path'] + "/" + fileName, fs_table[fileName]['new_path'] + "/" + fileName)
    #---------------------END

    #----DELETE_OLD_FOLDERS----
    for fileName in fs_table.keys():
        try:
            old_path = fs_table[fileName]['old_path']
            if old_path != dir_root:
                os.rmdir(old_path)
        except FileExistsError: 
            continue
    #--------------------------END
    
    #NOTE: return background agent output to user
    return answer

sort_FS_tool = client.create_tool(sort_FS)


In [97]:
sort_FS('_', 'content')

Testing Output: Using the template and given the following files, group the files into the template based on the commonalities of ONLY on the summary of the file.If an item has already be grouped, do not group it again. Thus, for each item listed, it should only appear once amoungst all groups. DO NOT CHANGE THE FILE NAMES. Return only the output template and nothing else:
Template:
[group 1]{item 1, item2, ...}
[group 2]{item 3, item 4, ...}
...
Items:
 file: "shoe.txt" content: "the manufacturing process for shoes"
 file: "tv.txt" content: "the manufacturing process for tv. "
 file: "fish.txt" content: "the life cycle for process for fishes."
 file: "bird.txt" content: "the life cycle for process for birds"
 file: "apple.txt" content: "cooking recipies for apples"
 file: "dog.txt" content: "the life cycle for process for dogs"
 file: "lemon.txt" content: "how to grow lemons"
MY_ANSWER "[Manufacturing Processes]{shoe.txt, tv.txt}\n[Life Cycles]{fish.txt, bird.txt, dog.txt}\n[Food Rela

FileNotFoundError: [Errno 2] No such file or directory: 'TestDirectory3/test1'

In [ ]:
file = open('dataTable3.json') #NOTE: hardCoded
fileData = json.load(file)
#fileDataS = json.dumps(fileData)

#create message to send to different agent
#NOTE: 1 layer simple sort, based on name
file_keys = fileData.keys()
message = ''
for k in file_keys:
    try:
        message =  message + '\n file: "' + k + '" content: "' + fileData[k]['summary'] + '"'
    except KeyError:
        continue
print(message)
    

In [19]:
agentName = "interfaceAgent"

agent_1 = client.create_agent(
    name=agentName,
    tools=[x.name for x in client.list_tools()]
    )